In [ ]:
import numpy as np
import pandas as pd
import tweepy 
import yahoo_fin.stock_info as si 
import schedule
import time
from datetime import date


today = date.today()
today = today.strftime("%d/%m/%Y")

In [ ]:
top_winners = si.get_day_gainers()
perc_gain = top_winners[(top_winners['% Change']<=10.0) & (top_winners['% Change']>=5) & (top_winners['Volume']//top_winners['Avg Vol (3 month)']>2)]
sorted_perc_gain = perc_gain.sort_values(by= 'PE Ratio (TTM)',ascending=True)
sorted_perc_gain = sorted_perc_gain.rename(columns={'Price (Intraday)':'Price'})
sorted_perc_gain['Price Out'] = np.nan
sorted_perc_gain['DateTime'] = today
short_list = sorted_perc_gain[['Symbol','Name','Price','% Change','Price Out','DateTime']]
short_list = short_list.reset_index(drop=True)
#short_list = short_list.set_index('Symbol')
dic_SL = short_list.to_dict(orient='index')
short_list.head()

In [ ]:
#gets env variables
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
CONSUMER_KEY = os.getenv('CONSUMER_KEY')
CONSUMER_SECRET = os.getenv('CONSUMER_SECRET')
ACCESS_TOKEN = os.getenv('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.getenv('ACCESS_TOKEN_SECRET')
BRR_TOKEN = os.getenv('BRR_TOKEN')

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [ ]:
Archive = pd.read_csv('Archive.csv',index_col=[0])
open_pos = Archive[Archive['Price Out'].isnull()]
lop = open_pos['Ticker'].to_list()
Archive.head()

In [ ]:

def tweet_and_archive(sl):
    ticker_l = []
    name_l = []
    price_l = []
    price_out_l = []
    date_time = []

    for index, row in sl.iterrows():
        print(sl.columns)
        Stock = row['Symbol']
        Price = row['Price']
        Price_out = row['Price Out']
        name_ = row['Name']
        Date_ = row['DateTime']
        if Stock not in lop:
            ticker_l.append(Stock)
            name_l.append(name_)
            price_l.append(Price)
            price_out_l.append(Price_out)
            date_time.append(Date_)
            bot_says = f'Possible swing of {Stock} at ${Price} long, This is not financial advice.'
            response = api.update_status(status=bot_says)
            print(response)
        else:
            continue

    to_add = pd.DataFrame({'Ticker':ticker_l,'Name':name_l,'Price':price_l,'DateTime':date_time,'Price Out':price_out_l})
    to_add['%Change'] = np.nan; 
    to_add['W/L'] = np.nan
    Arch = Archive.append(to_add,ignore_index=True)
    Arch.to_csv('Archive.csv')
#tweet_and_archive(short_list)

In [ ]:




while True:
    tweet_and_archive(short_list)
    time.sleep(86400)